In [2]:
# 회귀분석

# 회귀는 여러 개의 독립변수와 한 개의 종속변수 간의 상관관계를 모델링하는 기법을 통칭
# 머신러닝 회귀 예측의 핵심은 주어진 피처와 결정 값 데이터 기반에서 학습을 통해 최적의 회귀 계수를 찾아내는 것

# 회귀는 회귀 계수의 선형/비선형 여부, 독립변수의 개수, 종속변수의 개수에 따라 여러 가지 유형으로 나눌 수 있음

# 일반 선형 회귀 : 예측값과 실제값의 RSS를 최소화할 수 있도록 회귀계수를 최적화화며, 규제를 적용하지 않은 모델
# 릿지 : 선형 회귀에 L2 규제 추가
# 라쏘 : 선형 회귀에 L1 규제 적용
# 엘라스틱넷 : L2, L1 규제를 결합한 모델

In [4]:
# RSS

# 오류 값의 제곱을 구해서 더하는 방식
# 일반적으로 미분 등의 계산을 편리하게 하기 위해서 RSS 방식으로 오류 합을 구함

# 회귀식의 독립변수X, 종속변수 Y가 중심 변수가 아니라 w변수(회귀 계수)가 중심 변수임을 인지

# 비용함수, 손실함수
# 비용최소화 >> 경사하강법

# 다중공선성 문제

In [ ]:
# 회귀평가

# MAE : metrics.mean_absolute_error : 'neg_mean_absolute_error'
# MSE : metrics.mean_squared_error : 'neg_mean_squared_error'
# R제곱 : metrics.r2_socre : 'r2'